# The Remote Atomic Swap - Alice's code
#### 04.5 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-04

* Simple Atomic Swap: Alice send 2 Algos to Bob, and Bob send 1 Algos to Alice.
    * Usually, we would swap ALGOs for an ASA, this is just to simplify the example
* They are not in the same room.
* They exchange transactions via Email

### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [14]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
Alice  = cred['Alice']
Bob    = cred['Bob']

In [15]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn, SignedTransaction
import algosdk.error

import json
import pandas as pd
import base64

In [16]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

## The remote swap

#### Step 1: Prepare two transactions
* Alice prepares both transactions
* Similarly, a programmer could prepare the two transactions and send one to Alice and one to Bob

In [17]:
# Inspect the suggested_params
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))

{
    "first": 19787685,
    "last": 19788685,
    "gh": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
    "gen": "testnet-v1.0",
    "fee": 0,
    "flat_fee": false,
    "consensus_version": "https://github.com/algorandfoundation/specs/tree/bc36005dbd776e6d1eaf0c560619bb183215645c",
    "min_fee": 1000
}


In [18]:
algod_client.status()["last-round"]

19787686

##### How much time to do we have from creating the TX to signing and sending it?
* Algorand max window = 1000 blocks
* Already being proposed

In [20]:
# in rounds
print(sp.last - sp.first)
# in minutes (assuming 3 sec pro round)
print( (sp.last - sp.first)*3/60 )

1000
50.0


##### Possibly we want to extend the time for Bob to sign the transaction
* Start validity window later

In [29]:
sp.first = sp.first+10      # start later
sp.last = sp.last+10        # end later

In [23]:
amt_1 = int(2*1E6)
txn_1 = transaction.PaymentTxn(Alice["public"], sp, Bob["public"],  amt_1)

amt_2 = int(1*1E6)
txn_2 = transaction.PaymentTxn(Bob["public"], sp, Alice["public"],  amt_2)

#### Step 2: create and assign group id

In [24]:
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid

#### Step 3: Send transaction file to Bob
* Transaction `txn_2` is now ready and can be sent to Bob
* To be able to save it into a file, we need to `dictify` it

In [25]:
import pickle
data = txn_2.dictify()
file = open("Bob_txn.txt", 'wb')
pickle.dump(data, file)
file.close()

#### Step 4: Now it is Bob's turn
* We can assume that they exchange files via email or a similar service
* Open the notebook `04.5b_WSC` and create a file of the signed transaction

#### Step 5: Retrieve Bob's signed transaction
Instead of defining it as 'algosdk.future.transaction.PaymentTxn.undictify(...)', we use 'algosdk.future.transaction.SignedTransaction.undictify(...)'

In [26]:
file = open("Bob_signed.txt", 'rb')
data = pickle.load(file)
# To undictify, we need the SignedTransaction function
stxn_2 = SignedTransaction.undictify(data)

#### Step 6: Alice has to sign her transaction

In [27]:
stxn_1 = txn_1.sign(Alice['private'])

#### Step 7: Alice collects everything and sends the transaction
* This part could also be taken over by a third party

In [28]:
# Step 7.1: collect
signed_group =  [stxn_1, stxn_2]

# Step 7.2: send
txid = algod_client.send_transactions(signed_group)

# Step 7.3: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid) 

Current round is  19787793.
Waiting for round 19787793 to finish.
Waiting for round 19787794 to finish.
Transaction KDZWDUGGNIODWAWUEE4WT3XPOZ5XFZFN63L7ZYAXVNHSC5MFN7DA confirmed in round 19787795.
